# CIFAR10 Using Deep Learning

In [1]:
# Imports

%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from keras.datasets import cifar10

# Load the data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Reshape the images
# store_train = [row.reshape((32, 32, 3)) for row in X_train]
# store_test = [row.reshape((32, 32, 3)) for row in X_test]
# X_train = store_train
# X_test = store_test

/Users/saito/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using Theano backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [ ]:
X_train = np.swapaxes(X_train, 1, 2)
X_train = np.swapaxes(X_train, 2, 3)

X_test = np.swapaxes(X_test, 1, 2)
X_test = np.swapaxes(X_test, 2, 3)

In [ ]:
# Some helper functions and tools

# Batch iterator
class DataIterator(object):
    def __init__(self, *data, **params):
        self.data = data
        self.batchsize = params['batchsize']

    def __iter__(self):
        self.first = 0
        return self

    def __len__(self):
        return len(self.data[0])

    def __getitem__(self, key):
        outs = []
        for val in self.data:
            outs.append(val[key])
        return outs


class SequentialIterator(DataIterator):
    '''
    batchsize = 3
    [0, 1, 2], [3, 4, 5], [6, 7, 8]
    '''
    def next(self):
        if self.first >= len(self):
            raise StopIteration()
        outs = []
        for val in self.data:
            outs.append(val[self.first:self.first+self.batchsize])
        self.first += self.batchsize
        return outs

# One-hot vectorizer

def one_hot(vec, num_classes):
    to_return = np.zeros((vec.shape[0], num_classes))
    for idx, row in enumerate(vec):
        to_return[idx, row] = 1.0
    return to_return

y_train_onehot = one_hot(y_train, 10)
y_test_onehot = one_hot(y_test, 10)

In [ ]:
# Some global variables

x_dim = X_train[0].shape[1]
num_labels = 10
batch_size = 128
learning_rate = 1e-4
num_channels = 3
epochs = 10

## LeNet5

In [ ]:
# Placeholders
x_ph = tf.placeholder("float32", [None, x_dim, x_dim, num_channels])
y_ph = tf.placeholder("float32", shape=(None, num_labels))

# Weights and biases
# Conv layers
conv1_w = tf.Variable(tf.truncated_normal([5, 5, 3, 32],stddev=0.1,
                                         dtype="float32"))
conv1_b = tf.Variable(tf.zeros([32], dtype="float32"))

conv2_w = tf.Variable(tf.truncated_normal([5, 5, 32, 64],
                            stddev=0.1, dtype="float32"))
conv2_b = tf.Variable(tf.zeros([64], dtype="float32"))

# Fully-connected layers
fc1_w = tf.Variable(tf.truncated_normal([
            x_dim // 4 * x_dim // 4 * 64, 512
        ], stddev=0.1, dtype="float32"))
fc1_b = tf.Variable(tf.constant(0.1, shape=[512], dtype="float32"))

fc2_w = tf.Variable(tf.truncated_normal([
            512, num_labels
        ], stddev=0.1, dtype="float32"))
fc2_b = tf.Variable(tf.constant(0.1, shape=[num_labels],
                               dtype="float32"))

# Main model
conv1 = tf.nn.conv2d(x_ph, conv1_w, strides=[1, 1, 1, 1],
                    padding="SAME")
relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_b))
pool1 = tf.nn.max_pool(relu1, ksize=[1, 2, 2, 1],
                      strides=[1, 2, 2, 1], padding="SAME")

conv2 = tf.nn.conv2d(pool1, conv2_w, strides=[1, 1, 1, 1],
                    padding="SAME")
relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_b))
pool2 = tf.nn.max_pool(relu2, ksize=[1, 2, 2, 1],
                      strides=[1, 2, 2, 1], padding="SAME")

# pool_shape = pool2.get_shape().as_list()
pool_shape = tf.shape(pool2)
reshape = tf.reshape(
    pool2, 
    [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])

# Fully-connected layer
hidden1 = tf.nn.relu(tf.matmul(reshape, fc1_w) + fc1_b)
hidden2_logits = tf.matmul(hidden1, fc2_w) + fc2_b

# Output
output = tf.nn.softmax(hidden2_logits)

# Cross Entropy
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ph * tf.log(tf.clip_by_value(output, 
        1e-10, 1.0)), reduction_indices=[1]))

# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [ ]:
# Train

data = SequentialIterator(X_train, y_train_onehot, batchsize=batch_size)

init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

import time
t1 = time.time()

prev_error = 1000000
errors = []

for i in range(epochs):
    training_error = 0
    for x_batch, y_batch in data:
        feed_dict = {x_ph: x_batch, y_ph: y_batch}
        sess.run(optimizer, feed_dict=feed_dict)
        
        output = sess.run(cross_entropy, feed_dict=feed_dict)
        print output
        training_error += output
    
    print "Epoch %i\t, Training Error %f" % (i, training_error)
    if training_error > prev_error:
        break
    else:
        prev_error = training_error
    errors.append(training_error)

t2 = time.time()
print "Time taken to train: ", t2-t1, "seconds"

## CIFAR (Using Caffe)

In [ ]:
# Imports

import numpy as np
import caffe

solver = caffe.SGDSolver("cifar10_quick_solver.prototxt")
solver.solve()